In [20]:
import os
import pandas as pd
import math
import numpy as np
import tensorflow as tf

from datetime import datetime
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras import backend as K
from keras import regularizers
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.metrics import roc_auc_score

K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [21]:
datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split=0.2)

# The training set just has blanks instead of 0s
train_labels = pd.read_csv("../CheXpert-v1.0-small/train.csv").fillna(0)
train_labels["Path"] = '../' + train_labels["Path"]

In [22]:
# Filter out Lateral images.  We'll train two models -> one for lateral and one for frontal

frontal_train_labels = train_labels[train_labels['Frontal/Lateral'] == 'Frontal']

# Filter out uncertains in the training dataset.  There are no uncertains in the validation dataset.
frontal_train_labels = frontal_train_labels[frontal_train_labels["Pleural Effusion"] != -1.0]

In [23]:
frontal_train_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,../CheXpert-v1.0-small/train/patient00001/stud...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,../CheXpert-v1.0-small/train/patient00002/stud...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,../CheXpert-v1.0-small/train/patient00003/stud...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,../CheXpert-v1.0-small/train/patient00004/stud...,Female,20,Frontal,PA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,../CheXpert-v1.0-small/train/patient00005/stud...,Male,33,Frontal,PA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
frontal_validation_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,../CheXpert-v1.0-small/valid/patient64541/stud...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,../CheXpert-v1.0-small/valid/patient64542/stud...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,../CheXpert-v1.0-small/valid/patient64543/stud...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,../CheXpert-v1.0-small/valid/patient64544/stud...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,../CheXpert-v1.0-small/valid/patient64545/stud...,Female,55,Frontal,AP,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [25]:
train_datagen = datagen.flow_from_dataframe(dataframe=frontal_train_labels,
                                            directory=".",
                                            x_col="Path",
                                            y_col=['Pleural Effusion'],
                                            class_mode = "raw",
                                            color_mode='grayscale',
                                            target_size=(100, 100),
                                            batch_size=32,
                                           subset='training')
validation_datagen = datagen.flow_from_dataframe(dataframe=frontal_train_labels,
                                                directory=".",
                                                x_col="Path",
                                                y_col=['Pleural Effusion'],
                                                class_mode = "raw",
                                                color_mode='grayscale',
                                                target_size=(100, 100),
                                                batch_size=32,
                                                subset='validation')

Found 145160 validated image filenames.
Found 36289 validated image filenames.


In [26]:
# metrics functions

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [28]:
classifier = Sequential()

# 200x200 input
# Input is kinda large, so we select larger filters for the first layer to decrease 
# size of feature maps (and hopefully speed up training).

# Input: 100 x 100 x 1
classifier.add(Conv2D(32, (5, 5), input_shape=(100, 100, 1), use_bias=False))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 96 x 96 x 32
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 94 x 94 x 64
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 47 x 47 x 64
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 45 x 45 x 128
classifier.add(Conv2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 43 x 43 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 21 x 21 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 19 x 19 x 128
classifier.add(Conv2D(128, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(Dropout(.2))
# Input: 17 x 17 x 128
classifier.add(MaxPooling2D(pool_size=(2,2)))
# Input: 8 x 8 x 128
classifier.add(Flatten())

# Input: 8192 x 1024
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="relu", units=1024))
classifier.add(Dense(activation="sigmoid", units=1))

classifier.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=[precision_m, recall_m, f1_m])

mc = ModelCheckpoint('trial_1/weights{epoch:04d}.h5', save_weights_only=True, save_best_only=True, monitor='val_f1_m', mode='max')
# classifier.load_weights('trial_1/weights0068.h5')
os.makedirs('trial_1')

# too many epochs mean overfitting, not enough epochs mean underfitting
classifier.fit_generator(
    train_datagen,
    steps_per_epoch=5280,
    epochs=120,
    initial_epoch=0,
    validation_data=validation_datagen,
    validation_steps=800,
    workers=4,
    verbose=2,
    callbacks=[mc])

Epoch 1/120
 - 1811s - loss: 0.5744 - precision_m: 0.6599 - recall_m: 0.6278 - f1_m: 0.6253 - val_loss: 0.6613 - val_precision_m: 0.5246 - val_recall_m: 0.9108 - val_f1_m: 0.6597
Epoch 2/120
 - 1811s - loss: 0.5162 - precision_m: 0.7082 - recall_m: 0.7001 - f1_m: 0.6943 - val_loss: 0.7588 - val_precision_m: 0.4770 - val_recall_m: 0.9693 - val_f1_m: 0.6337
Epoch 3/120
 - 1813s - loss: 0.5001 - precision_m: 0.7162 - recall_m: 0.7185 - f1_m: 0.7083 - val_loss: 0.7744 - val_precision_m: 0.5500 - val_recall_m: 0.9200 - val_f1_m: 0.6818
Epoch 4/120


KeyboardInterrupt: 